# Crawler Noticias



## Introdução

Estudo projeto de construção de um bot crawler para extrair e indexar noticias de sites.

É recomendavel baixar uma única vez o html do site para reduzir a necessidade de acessar o site várias vezes para cada processamento reduzindo tempo e custo computacionais para máquina local e servidor.

Usamos módulo _Requests_ para baixar o html da página a ser processada e com módulo _BeautifulSoup_ processamos o conteúdo html buscando por conteúdos especificos. Ao inspecionar o código HTML verificamos em quais blocos estão as principais noticias - em geral em tags como "section", "h1"/"h2"/"h3" e "a" - considerando o seletor css que especifica cada bloco de interesse afim de refinar a busca e garantir que serão extraindos os trechos relevantes.

Sites com boas práticas de HTML/CSS tendem a serem mais faceis de serem processados e indexados por mecanismos de buscas facilitando os usuários em encontrar seus conteúdos em pesquisas aumentando assim o engajamento em suas plataformas contribuindo para seu melhor desenvolvimento.

Neste presente momento o projeto está considerando três portais de notícias que são _BBC Brasil_, _CNN Brasil_ e _G1/Globo_. 

Créditos e direitos reservados às referentes plataformas mencionadas.

(Obs.: Conteúdo livre com fins informativos e de divulgação. )


## Módulos

Abaixo _import_ dos módulos que contém os recursos usados.

* BeautifulSoup
* Requests
* re (regular expression)
* crawlernewsg1 (acessa site e extrai lista de principais noticias na página principal do site)
* IPython.core.display -> display, HTML (renderiza conteúdos html )

In [14]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from crawlernewsg1 import *
import random

# Render html content
from IPython.core.display import display, HTML

C:\Users\wsric\AppData\Local\Temp\ipykernel_17036\3045104897.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Modelagem Portal G1



Capturando noticias do portal G1 da Globo
Funções básicas. Especifincando "selector css" classe 'bastian-page'  para tags div.

In [15]:
# Baixando html do portal de noticia para processamento e salvando em disco.
url = 'https://g1.globo.com'
attr = {'class': 'bastian-page'}
data = g1_(requests.get( url ).content, 'div', attr, debug=False)
dw = requests.get('https://g1.globo.com/').content
with open('tmp/data', 'w', encoding="utf-8") as fl:
    fl.write( dw.decode( ) )
    

In [ ]:
#dw.decode()

In [17]:
type(data)

tuple

In [18]:
# Replace field name 'titulo' for 'title' name.
#dw = [ { key.replace('titulo', 'title'):value for key, value in i.items() } for i in dw ]
#dw = [ { key.replace('url', 'href'):value for key, value in i.items() } for i in dw ]
#replKey = lambda listdc, old_key, new_key: [ ]
def replKey(listdc, old_key, new_key  ):
    dt = []
    
    
    for i in listdc:
        for keys, value in i.items():
            dt.append( { keys.replace('url', 'href'): value } ) 
            
    return dt

data = replKey(data[0], 'url', 'href' )

print(data)

[{'title': 'Cidade engolida por rio revive pesadelo com novo temporal; veja vídeo'}, {'href': 'https://g1.globo.com/rs/rio-grande-do-sul/noticia/2024/05/10/video-cidade-que-ficou-submersa-no-vale-do-taquari-revive-pesadelo-com-novos-temporais.ghtml'}, {'title': 'ASSISTA: Ajuda Rio Grande mostra alagamentos no sul do estado'}, {'href': 'https://g1.globo.com/rs/rio-grande-do-sul/ao-vivo/tragedia-no-rio-grande-do-sul.ghtml'}, {'title': 'Governo explica regra para ajuda emergencial a prefeitos e critica distorções'}, {'href': 'https://g1.globo.com/politica/noticia/2024/05/10/ministros-falam-sobre-auxilio-do-governo-ao-rio-grande-do-sul.ghtml'}, {'title': 'É #FAKE que vídeo mostre descarte de alimentos doados ao RS'}, {'href': 'https://g1.globo.com/fato-ou-fake/noticia/2024/05/10/e-fake-que-video-mostre-descarte-de-alimentos-doados-para-vitimas-dos-temporais-no-rs.ghtml'}, {'title': 'Guaíba recebeu metade do volume da 2ª maior hidrelétrica do mundo'}, {'href': 'https://g1.globo.com/meio-amb

In [19]:
dw, cwn = g1_( dw, 'div', attr)

Ao abrir link da noticia pesquisar pela tag *'p'* com atributos **class** com valor **"content-text__container** definir o tamanho para caso extrair só parte do corpo do texto.


Retornando uma lista dos itens encontrados (como visto acima no código) pegamos estes itens e os concatenamos exibintido o texto no corpo da noticia. (_Como visto abaixo_)

Cada "_evt" (_css selector_ class) class css em "bastian-page" refere-se a uma noticia na lista central de noticias.
Dentro de cada "_evt" haverá "bastian-feed-item" e neste o feed-post. 

**feed-post-body** _contêm_  ( 'feed-post-link', 'feed-post-body-title', 'feed-post-body-resumo')

**bastian-feed-item** _contem_ um feed-post-body referindo-se a cada item (noticia)


Para link da noticia (quando acessando a noticia)

**content-head__title** em tag 'h1' (Título da noticia)

**content-head__subtitle** em tag 'h2' (subtitulo/resumo da noticia)

**content-text__container** corpo do texto da noticia css-selector, tag 'p' (pegar só a primeira referente ao primeiro paragrafo da noticia)

In [20]:
news2 = '<h1 style="padding: 12px;">Notícias</h1>'
news2 += '<br><br>'.join( [ '<br>'.join( [ str( i['title'] ) , str( i[ 'url' ]  ) ] ) for i in dw ] )
display( HTML( news2 ))

## CNN Crawler de Noticias do Portal

In [21]:
cnn_data = requests.get('https://www.cnnbrasil.com.br/')


In [22]:
cnn_soup = BeautifulSoup(cnn_data.content, 'html.parser')
cnn_nw_data = cnn_soup.find_all('section')
cnn_nw_data[0]

<section class="theater dark"> <div class="container"> <div class="row"> <div class="col--3 col__l--2"> <div class="desc"> <img alt="Prime Time" class="thumb-program" src=""/> <p class="descr-program"> </p> </div> <div class="date-time-program"> <p>seg - sex</p> <p></p> </div> <div class="presenter"> <p>Apresentação</p> <p></p> </div> </div> <div class="col--7 col__l--8"> <p class="title-ao-vivo">Ao vivo</p> <div class="theater__video"> <div class="post__video"> <div class="cnn component-video has--live"> <div class="component-inner"> <div class="video-wrapper" id="player_xDNzUATybvw"> <div class="video-wrapper-inner"> <div class="overlay-wrapper"> <button aria-label="Assista o vídeo Home" class="video-button js-video-play active" tabindex="0"> <img alt="Home" class="thumbnail-image" src="https://img.youtube.com/vi/xDNzUATybvw/sddefault.jpg"> <span aria-label="Botão play do vídeo Home" class="video-play-button"></span> <span class="video-gradient"></span> <span class="video-title">Home

In [23]:
c = cnn_nw_data[0].find_all('a')
#dir(c)


In [24]:
for i in c:
    print( i )

In [25]:
cnn_list_news = []
aux = None 
for news in cnn_nw_data:
    aux = news.find('a')
    
    
    try :
        
        aux = aux.attrs
        cnn_list_news.extend( [ { 'title': aux['title'], 'href': aux['href'] } ] ) 

    except:
        pass

for i in cnn_list_news:
    print(cnn_list_news)

[{'title': 'Com alerta de mais chuvas no RS, Leite apela para população não voltar às áreas de risco', 'href': 'https://www.cnnbrasil.com.br/nacional/tragedia-no-rs-com-previsao-de-mais-chuvas-governador-apela-para-que-populacao-nao-volte-as-areas-de-risco/'}]


In [26]:
cnn_list_news_ma = None
for i in cnn_list_news:
    
    try:
        print(f"\n{i['title']} \n{i['href']}\n\n" ) 
    except:
        pass 
    #keys = i.keys()
    #print(keys)   
    

    #print( f'\t\n\n{out}') 
    #out = '' 



Com alerta de mais chuvas no RS, Leite apela para população não voltar às áreas de risco 
https://www.cnnbrasil.com.br/nacional/tragedia-no-rs-com-previsao-de-mais-chuvas-governador-apela-para-que-populacao-nao-volte-as-areas-de-risco/




In [27]:
len(dw)

8

In [28]:
len(cnn_list_news)

1

## BBC Brasil 

Crawler das notícias do portal do site BBC Brasil.

In [41]:
url_bbc = 'https://www.bbc.com/portuguese'
url_bbc_base = 'https://www.bbc.com'

In [42]:
bbc_data = requests.get(url_bbc)
bbc_soup = BeautifulSoup(bbc_data.content, 'html.parser')
bbc_sections =  bbc_soup.find_all('section' )

In [43]:
bbc_news_lists = []

``` python
url_bbc_base+bbc_sections[0].select('a')[0]['href']
```

Saída 'https://www.bbc.com/portuguese/brasil-63507138'

In [61]:
bbc_sections[0].select('a')

[]

In [62]:
bbc_sections[1].select('p')[0]

<p class="promo-paragraph bbc-1xza832 ewjbyra0">Nos próximos dias, seguradoras terão de pagar quantia recorde em sinistros — mas nem o próprio setor sabe estimar ainda quanto será esse valor.</p>

In [ ]:
for news in bbc_sections:
    bbc_news_lists.extend( [ { 'title': i.text, 'url': i['href'] } for i in news.select('a') ] )

In [ ]:
bbc_news_lists = []
aux = None 

for section_news in bbc_sections:
    for news in section_news.select( 'a' ):
        if news['href'].find('topic') != -1:
            pass
        else :
            if news['href'].find('https') != -1:
                bbc_news_lists.extend( [ { 'title': news.text, 'href': news['href'] } ] )
            else:
                bbc_news_lists.extend( [ { 'title': news.text, 'href': url_bbc_base+news['href'] } ] )

In [ ]:
print(f'Número de notícias: {len(bbc_news_lists)}')

In [ ]:
random.choices(bbc_news_lists, k = 3)

In [ ]:
print(url_bbc_base+'/portuguese/topics/cz74k71p8ynt')

In [ ]:
with open('bbc_news.txt', 'w') as fl:
    for i in bbc_news_lists:
        fl.write(i['title'] +'\n' + i['href'] +'\n\n')
    fl.close()

## Portal Band

Busca de lista de notícias do portal Band.

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url_band = 'https://www.band.uol.com.br/'

In [ ]:
band_data = requests.get(url_band).content
soup_band = BeautifulSoup(band_data, 'html.parser')


#### Band list sections 

In [ ]:
news_band_section = soup_band.find_all('section')

In [ ]:
print(news_band_section[0])
print(f'Número: {len(news_band_section)}')

#### Band home main

![bandcapt.png](bandcapt.png)

1. tag main class -> home
2. Tag section com class=hardnews
3. tag div's conteúdos com para classes hardnews__highlight e cards

In [ ]:
import re
# Tag 'main' e class=[home container]. (variável band_main_home)
band_main_home = soup_band.find_all('main', class_='home')
# -----------------------
#dir(band_main_home[0] )
#print(str( band_main_home ) )
soup_band2 = BeautifulSoup( str( band_main_home ) , 'html.parser')

bmainhardnews = soup_band2.find_all('section', 'hardnews')
soup_blocknews = BeautifulSoup( str( bmainhardnews[0] ), 'html.parser' )
bn1 = soup_blocknews.find_all( 'div', class_='hardnews__highlight' )

soup_links_news = BeautifulSoup(str( bn1 ), 'html.parser') 
bn1_link_news = soup_links_news.find_all( 'a'  )#, class_= [ 'link image','related', 'link']  ) 
bn2 = soup_blocknews.find_all( 'div', class_='cards' )

soup2bn2 = BeautifulSoup( str( bn2[0] ), 'html.parser' ) 
bn2_cards_links = soup2bn2.find_all( 'a' )

# Tag: div Class -> hardnews__highlight and card. Duas seções/blocos de noticias.
#print('\n> bn1 ', bn1[0] )
#print('\n> bn2 ', bn2[0] )
print('\n bn1_link_news ', len(bn1_link_news ))
print('\n bn1_link_news ', bn1_link_news[1] )
print('\n'+30*'-')
print('\n bn2_cards_links (size)', len(bn2_cards_links) )
print('\ bn2_cards_links ', bn2_cards_links[-1] ) #Pegando o último elemento da lista e exibindo.


In [ ]:
#print(dir(bn1_link_news[0] ) )
#bn1_link_news[0]['href']

# Lista de links de notícias no bloco {tag: div, class_: 'hardnews__highlight' } do HTML.
for i in bn1_link_news:
    print('\n> ', i.text +'\n | '+i['href'])

In [ ]:
for i in bn2_cards_links:
    print('\n> ', i['title'] +'\n | '+i['href'])

#### Função "trackBandNews" (Obtendo notícias do portal da Band)

In [ ]:
# Type list of dictionary
ListDict = list[ dict ]

def trackBandNews(data: str) -> ListDict:
    """
    data: str [dados] text html
    return list of news with fields 'Title of news' and 'url' for news.
    l = [
    {'title': 'string', 'href': link},
    ...
    ]
    
    """
    # Lista de notícias
    result = []
    
    # Obtendo conteúdo html
    #band_data = requests.get(url_band).content
    soup_band = BeautifulSoup(data, 'html.parser')
    
    # Primeiro acesso tag main (bloco principal que contêm conteúdos como as noticias principais e seus links.
    # Tag 'main' e class=[home container]. (variável band_main_home)
    band_main_home = soup_band.find_all('main', class_='home')
    
    # Capturando html do bloco maior em main que contêm as noticias.
    soup_band2 = BeautifulSoup( str( band_main_home ) , 'html.parser')
    
    # Este block ( tag:section, class:hardnews) filho da tag main contem dois blocos de noticias a serem tratados insoladamente.
    bmainhardnews = soup_band2.find_all('section', 'hardnews')
    soup_blocknews = BeautifulSoup( str( bmainhardnews[0] ), 'html.parser' )
    bn1 = soup_blocknews.find_all( 'div', class_='hardnews__highlight' )

    
    soup_links_news = BeautifulSoup(str( bn1 ), 'html.parser') 
    bn1_link_news = soup_links_news.find_all( 'a'  )#, class_= [ 'link image','related', 'link']  ) 
    bn2 = soup_blocknews.find_all( 'div', class_='cards' )

    soup2bn2 = BeautifulSoup( str( bn2[0] ), 'html.parser' ) 
    bn2_cards_links = soup2bn2.find_all( 'a' )

    # Tag: div Class -> hardnews__highlight and card. Duas seções/blocos de noticias.
    #print('\n> bn1 ', bn1[0] )
    #print('\n> bn2 ', bn2[0] )
    #print('\n bn1_link_news ', len(bn1_link_news ))
    #print('\n bn1_link_news ', bn1_link_news[1] )
    #print('\n'+30*'-')
    #print('\n bn2_cards_links (size)', len(bn2_cards_links) )
    #print('\ bn2_cards_links ', bn2_cards_links[-1] ) #Pegando o último elemento da lista e exibindo.
    
    # Lista de links de notícias no bloco {tag: div, class_: 'hardnews__highlight' } do HTML.
    for i in bn1_link_news:
        result.append( { 'title': i.text, 'href': i['href'] } )
        #print('\n> ', i.text +'\n | '+i['href'])
        
    for i in bn2_cards_links:
        result.append( { 'title' : i['title'], 'href':  i['href'] } )
        #print('\n> ', i['title'] +'\n | '+i['href'])
    
    return result

In [ ]:
band_news = trackBandNews( band_data )
band_news

#### Percorrer listas de elementos em 'news_band_section'

In [ ]:
aux = []
list_ = None
for secs in news_band_section:
    list_ = secs.find_all('a', class_='link')
    for i in list_:
        
        try:
            aux.extend( [ { 'title': i.h3.text+'. '+ i.h2.text, 'href': i['href']} ] )
        except:
            aux.extend( [ { 'title': i.text, 'href': i['href'] } ] )
    #aux.extend([ { 'title': i}  for i in list_ ]  )
news_list_band = [] 
news_list_band.extend(random.choices(aux, k=4 ) )

In [ ]:
n = soup_band.find_all('a')

In [ ]:
import random

random.choices(news_list_band, k = 4)[0:]
for i in news_list_band:
    print(f"\n{i['title']}\n{i['href']}")

In [ ]:
n[0]

In [ ]:
#print(dir(n[0]))
n[0]

In [ ]:
# Lista de links possivelmente de notícias no site da Band.
for i in n:
    try:
        print(i['href'])
    except:
        print("no link")

## Folha de São Paulo 

Lendo feed RSS do site de notícias da [Folha de São Paulo](https://www1.folha.uol.com.br/feed/).

**Editoriais** Disponíveis

- [Em Cima Da Hora](https://feeds.folha.uol.com.br/emcimadahora/rss091.xml)
- [Opinião](https://feeds.folha.uol.com.br/opiniao/rss091.xml)
- [Política](https://feeds.folha.uol.com.br/poder/rss091.xml)
- [Mundo](https://feeds.folha.uol.com.br/mundo/rss091.xml)
- [Mercado](https://feeds.folha.uol.com.br/mercado/rss091.xml)
- [Cotidiano](https://feeds.folha.uol.com.br/cotidiano/rss091.xml)
- [Educação](https://feeds.folha.uol.com.br/educacao/rss091.xml)
- [Equilíbrio](https://feeds.folha.uol.com.br/equilibrio/rss091.xml)
- [Esporte](https://feeds.folha.uol.com.br/esporte/rss091.xml)
- [Ilustrada](https://feeds.folha.uol.com.br/ilustrada/rss091.xml)
- [Ilustríssima](https://feeds.folha.uol.com.br/ilustrissima/rss091.xml)
- [Ciência](https://feeds.folha.uol.com.br/ciencia/rss091.xml)
- [Ambiente](https://feeds.folha.uol.com.br/ambiente/rss091.xml)
- [Tec](https://feeds.folha.uol.com.br/tec/rss091.xml)
- [Comida](https://feeds.folha.uol.com.br/comida/rss091.xml)
- [Saúde](https://feeds.folha.uol.com.br/equilibrioesaude/rss091.xml)
- [Folhinha](https://feeds.folha.uol.com.br/folhinha/rss091.xml)
- [Turismo](https://feeds.folha.uol.com.br/turismo/rss091.xml)


In [ ]:
import feedparser as fp

fsp_editoriais = [ 
    ('Em Cima Da Hora', 'https://feeds.folha.uol.com.br/emcimadahora/rss091.xml'),
    ('Opinião', 'https://feeds.folha.uol.com.br/opiniao/rss091.xml' ) ,
    ( 'Política', 'https://feeds.folha.uol.com.br/poder/rss091.xml') ,
    ('Mundo', 'https://feeds.folha.uol.com.br/mundo/rss091.xml'),
    ('Mercado','https://feeds.folha.uol.com.br/mercado/rss091.xml'),
    ('Cotidiano','https://feeds.folha.uol.com.br/cotidiano/rss091.xml'),
    ('Educação', 'https://feeds.folha.uol.com.br/educacao/rss091.xml'),
    ('Equilíbrio', 'https://feeds.folha.uol.com.br/equilibrio/rss091.xml'),
    ('Esporte','https://feeds.folha.uol.com.br/esporte/rss091.xml'),
    ('Ilustrada', 'https://feeds.folha.uol.com.br/ilustrada/rss091.xml'),
    ('Ilustríssima','https://feeds.folha.uol.com.br/ilustrissima/rss091.xml'),
    ('Ciência','https://feeds.folha.uol.com.br/ciencia/rss091.xml'),
    ('Ambiente','https://feeds.folha.uol.com.br/ambiente/rss091.xml'),
    ('Tec','https://feeds.folha.uol.com.br/tec/rss091.xml'),
    ('Comida','https://feeds.folha.uol.com.br/comida/rss091.xml'),
    ('Saúde','https://feeds.folha.uol.com.br/equilibrioesaude/rss091.xml'),
    ('Folhinha','https://feeds.folha.uol.com.br/folhinha/rss091.xml'),
    ('Turismo','https://feeds.folha.uol.com.br/turismo/rss091.xml'),
]

In [ ]:
# Lista dos feeds pelos editoriais.
#l = [ {i: {'title': j.title, 'href': j.link } } for i,j in zip(range(len(d.entries) + 1), d.entries) 
#fsp_feeds = [ [ { i: { 'title': j.title, 'href': j.link } } for i, j in zip( range( len( fp.parser(feed) ) ), feed.entries ) ]  for feed in fsp_editoriais ]
fsp_feeds = [ 
    { i: ( j[0], fp.parse( j[1] ) ) }
    for i,j in zip( range( len( fsp_editoriais) + 1 ), fsp_editoriais ) 
]




In [ ]:
fsp_feeds2 =  [ 
    { 'Editorial': j[0], 'FeedData': fp.parse( j[1] ) } 
    for j in fsp_editoriais  
]

In [ ]:
# Lista de editoriais com suas respectivas listas de notícias. Para cada editorial extraido um lista de notícias.
fsp_news_list = [
    { 
        'Editorial': i[ 'Editorial' ],
        'Noticias': [ { 
            'Title': j.title,
            'Summary': j.summary,
            'href': j['link'].split('*')[1]
     }  for j in i[ 'FeedData' ].entries ]
    } for i in fsp_feeds2 
]

In [ ]:
def trackFolhaSP():
    """
    Track página da "Folha de São Paulo"
    """
    fsp_editoriais = ( 
            ('Em Cima Da Hora', 'https://feeds.folha.uol.com.br/emcimadahora/rss091.xml'),
            ('Opinião', 'https://feeds.folha.uol.com.br/opiniao/rss091.xml' ) ,
            ( 'Política', 'https://feeds.folha.uol.com.br/poder/rss091.xml') ,
            ('Mundo', 'https://feeds.folha.uol.com.br/mundo/rss091.xml'),
            ('Mercado','https://feeds.folha.uol.com.br/mercado/rss091.xml'),
            ('Cotidiano','https://feeds.folha.uol.com.br/cotidiano/rss091.xml'),
            ('Educação', 'https://feeds.folha.uol.com.br/educacao/rss091.xml'),
            ('Equilíbrio', 'https://feeds.folha.uol.com.br/equilibrio/rss091.xml'),
            ('Esporte','https://feeds.folha.uol.com.br/esporte/rss091.xml'),
            ('Ilustrada', 'https://feeds.folha.uol.com.br/ilustrada/rss091.xml'),
            ('Ilustríssima','https://feeds.folha.uol.com.br/ilustrissima/rss091.xml'),
            ('Ciência','https://feeds.folha.uol.com.br/ciencia/rss091.xml'),
            ('Ambiente','https://feeds.folha.uol.com.br/ambiente/rss091.xml'),
            ('Tec','https://feeds.folha.uol.com.br/tec/rss091.xml'),
            ('Comida','https://feeds.folha.uol.com.br/comida/rss091.xml'),
            ('Saúde','https://feeds.folha.uol.com.br/equilibrioesaude/rss091.xml'),
            ('Folhinha','https://feeds.folha.uol.com.br/folhinha/rss091.xml'),
            ('Turismo','https://feeds.folha.uol.com.br/turismo/rss091.xml'),
        )

    fsp_feeds =  ( 
            { 'Editorial': j[0], 'FeedData': fp.parse( j[1] ) } 
                for j in fsp_editoriais  
        )  

    # Lista de editoriais com suas respectivas listas de notícias. Para cada editorial extraido um lista de notícias.
    fsp_news_list = (
            { 
                'Editorial': i[ 'Editorial' ],
                'Noticias': ( { 
                    'Title': j.title,
                    'Summary': j.summary,
                    'href': j['link'].split('*')[1]
            }  for j in i[ 'FeedData' ].entries )
            } for i in fsp_feeds 
        )

    return  fsp_news_list 

In [ ]:
# Testes dos algoritmos

In [ ]:
f = trackFolhaSP()


In [ ]:
import sys

In [ ]:

sys.getsizeof(f)

In [ ]:
sys.getsizeof( fsp_news_list )

In [ ]:
for i in f:
    print(i)

In [ ]:
for j in fsp_news_list:
    print(j)

In [ ]:
fsp_feeds[0][0]

In [ ]:
fsp_editoriais

## Montando Lista de Notícias

In [ ]:
number_news = 6


In [ ]:
import random

In [ ]:
news_list = []

news_list.extend( [ {'title': random.choices( [ (news['title'], news['url']) for news in dw ] , k = 2 ), 'source': 'G1/Globo' } ] )

news_list.extend( [ {'title': random.choices( [ (news['title'], news['href']) for news in cnn_list_news ] , k = 2 ), 'source': 'CNN Brasil' } ] )

news_list.extend( [ {'title': random.choices( [ (news['title'], news['href']) for news in bbc_news_lists ] , k = 2 ), 'source': 'BBC Brasil' } ] )

news_list.extend( [ { 'title':  random.choices( [ ( news['title'], news['href'] ) for news in band_news  ] , k= 2 ), 'source': 'Band' } ] )


In [ ]:
for i in news_list:
    for j in i['title']:
        print(f"{j[0]}. \n{j[1]}\n")
    
    print(f"Fonte: {i['source']}\n\n")

## Salvando Todas Noticias (JSON)

### View news for test algorithm

In [ ]:
# List news
news = None
portaisvar = [
    'dw', #Globo/G1
    'cnn_list_news', # CNN Brazil
    'bbc_news_list' # BBC Brazil
]

portais = { 'dw': 'Globo/G1', 'cnn_list_news': 'CNN Brasil', 'bbc_news_lists': 'BBC Brasil' }    

In [ ]:
dw[0].keys(), cnn_list_news[0].keys(), bbc_news_lists[0].keys()

In [ ]:
# Replace field name 'titulo' for 'title' name.
dw = [ { key.replace('titulo', 'title'):value for key, value in i.items() } for i in dw ]
dw = [ { key.replace('url', 'href'):value for key, value in i.items() } for i in dw ]
dw

In [ ]:
#[ { key.replace('title', 'titulo' ) } ]
newslist = dw + cnn_list_news + bbc_news_lists
#newslist

In [ ]:
import os
count = 0
try:
    os.mkdir( 'newsdata' )
except FileExistsError:
    print("File exist")
    
webdata = ''
# Save data of link news in files to folder.
for news in newslist:
    try:
        #news['data'] = requests.get(news['href']).text
        foldername = f'newsdata'
        #os.mkdir( foldername )
        news['datafile'] = f'{foldername}/'+str( count )
        with open( foldername+f'/{str(count)}', 'a' ) as fl:
            webdata = requests.get( news['href'] ).text 
            fl.write( webdata )
            fl.close()
            
        count += 1
    except :
        print('not get page')
        continue
    

In [ ]:
newslist[0]['datafile']

In [ ]:
p = requests.get( newslist[0]['href'] )
soup = BeautifulSoup( p.text, 'html.parser')
texto = soup.get_text()
print( texto[0:150] )

In [ ]:
d = ''
with open(foldername+'/0', 'r') as fl:
    d = fl.read()
    fl.close()


soup = BeautifulSoup(d, 'html.parser')
#souptext = soup.text
content = soup.find_all('div')
#dir(soup)

In [ ]:
texto = ' '.join( [ i.get_text() for i in content ] )
texto = texto.replace('\n', ' ')
with open('texto', 'w' ) as fl:
    #fl.write( souptext )
    fl.write( texto )
    

In [ ]:
print( texto[0:100] )

In [ ]:
import json

with open('noticias.json', 'w') as fl:
    fl.write(json.dumps(newslist, ensure_ascii=False, indent=4))
    fl.close()

### Salva noticias

In [ ]:
def savenews(newslist: list, filename: str) -> None:
    import json
    jsondata = None
    with open(filename, 'w') as fl:
        fl.write( json.dumps( newslist, encode='utf-8', indent=4 ) )
        fl.close()
    print('Saved...')
    

In [ ]:
i = j = None

for i in news_list:
    for j in i['title']:
        print(f'{j[0]}.')
    print(f'Fonte: {i["source"] }\n\n ' )

In [ ]:
i = j = None

print(f"Olá bem vindo ao Diário de Notícias Dimensão Alfa. Estas são as principais manchetes do dia.\n")
for i in news_list:
    print(f'Portal de Notícias {i["source"] }\n ' )
    for j in i['title']:
        print(f'{j[0]}.')

# Body Texts

# Sobre

## Dimensão Alfa

Dimensão Alfa projetos e conteúdos de tecnologia.

## Info

O presente projeto tem sido usado com fins de divulgação e facilitação de acesso a noticias e conhecimento em comunhão com objetivo da plataforma/página Dimensão Alfa. 
Conteúdos de terceiros são de responsabilidades dos mesmos bem como seus direitos autorais.

O projeto encontra-se em desenvolvimento, inicialmente fôra batizado de Ani Fátima Liu, e estará passando por alterações estando de inicio disponibilizado em formato "_jupyter notebook_" podendo servir como _case_ de estudo para os que se interessam por "web scrap" (raspagem de dados).

Tecnologias foram usadas para gerar vídeo de noticias diária para página [Youtube](https://www.youtube.com/@dimensaoalfa); foi usada as seguintes tecnologias:

* [Editor de códigos VSCode](https://code.visualstudio.com/)
* [Python (linguagem de programação)](https://www.python.org/)
* [Ambiente JupyterLab](https://jupyter.org/)
* [Biblioteca "Requests"](https://requests.readthedocs.io/en/latest/)
* [Biblioteca "BeautifulSoup"](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
* [Serviço de Sintese de Voz Microsoft/Azure](https://speech.microsoft.com)


Peço e agradeço a compreensão e apoio de todos. 

Para contribuições, dúvidas, sugestões visitem meu blog [WSRicardo](https://wsricardo.blogspot.com).


## Sugestões de Conteúdo

Deixamos abaixo algumas sugestões de conteúdos e canais com recursos para estudos e pesquisa que podem ser uteis para quem se interessa por tecnologia, programação de computadores, matemática, ciências de dados e inteligência artificial.

* [Programação Dinâmica](https://www.youtube.com/c/Programa%C3%A7%C3%A3oDin%C3%A2mica)
* [Toda Matemática](https://www.youtube.com/c/GustavoViegascurso)
* [Matemática Universitária](https://www.youtube.com/c/Matem%C3%A1ticaUniversit%C3%A1riaProfRenan)
* [Reflexões Matemáticas](https://www.youtube.com/c/Reflex%C3%B5esMatem%C3%A1ticasDrDilbertoJ%C3%BAnior)
* [Programação Descomplicada](https://www.youtube.com/user/progdescomplicada)
* [Univesp](https://www.youtube.com/user/univesptv)
* [USP no Youtube](https://www.youtube.com/c/CanalUSP)
* [IME/USP](https://www.ime.usp.br/)
* [IMPA](https://www.youtube.com/c/impabr)



## Links

* [Dimensão Alfa](https://www.dimensaoalfa.com.br)
* [Facebook](https://www.facebook.com/dimensaoalfa)
* [Youtube](https://www.youtube.com/@dimensaoalfa)
* [WSRicardo](https://wsricardo.blogspot.com)